In [1]:
# Libraries and dependencies
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.integrate import quad

In [2]:
# OIS
ois = pd.read_pickle('Odisc')
# FSR
fsr = pd.read_pickle('FSR')
# SABR
SABRdf = pd.read_pickle('SABRdf')

In [3]:
# Black76 model
def Black76(F, K, vol, T, disc, Type = 'Payer'):
    c1 = (np.log(F/K) + (vol**2)/2*T) / (vol*np.sqrt(T))
    c2 = c1 - vol*np.sqrt(T)
    if Type == 'Payer':
        return disc*(F*norm.cdf(c1) - K*norm.cdf(c2))
    elif Type == 'Receiver':
        return disc*(K*norm.cdf(-c2) - F*norm.cdf(-c1))

# SABR model
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

### Question 1
Given a decompounded option pays the following at time $T = 5y$:

$$
\begin{align*}
\text{CMS}\ 10y^{\frac{1}{4}} - 0.04^{\frac{1}{2}}
\end{align*}
$$

For static replication of any constant maturity swap (CMS) payoff $g(F)$, where $F$ is the swap rate, we use the following formula:

  \begin{equation*}
    \begin{split}
      V_0 &= D(0,T) g(F) + h'(F)[V^{pay}(F)-V^{rec}(F)] \\
      &\;\;\;\;\;\;\;\;\;\;+ \int_0^F h''(K) V^{rec}(K) dK +
      \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

Let $F = \text{CMS}\ 10y$ and $g(K)$ be the payoff function. Differentiating, we get:

$$
\begin{align*}
g(K) &= F^{\frac{1}{4}} - 0.2 \\
g'(K) &= \frac{1}{4} F^{-\frac{3}{4}} \\
g''(K) &= -\frac{3}{16} F^{-\frac{7}{4}}
\end{align*}
$$

In [4]:
def g_0(K):
    return K**(1/4)-0.2

def g_1(K):
    return 1/4*K**(-(3/4))

def g_2(K):
    return -(3/16)*K**(-(7/4))

Let $\text{IRR}(K)$ be the IRR annuity. Differentiating, we get:

\begin{equation*}
\begin{split}
\mbox{IRR}(K)&=\sum_{i=1}^{N\times m}\frac{1}{(1+\frac{K}{m})^i}=\frac{1}{K}\left[1-\frac{1}{\left(1+\frac{K}{m}\right)^{N\times m}}\right]\\
\mbox{IRR}'(K)&=-\frac{1}{K}\mbox{IRR}(K)
+\frac{1}{m\times K}\frac{N\times m}{\left(1+\frac{K}{m}\right)^{N\times m+1}} \\
\mbox{IRR}''(K)&=-\frac{2}{K}\mbox{IRR}'(K)
-\frac{1}{m^2\times K}\frac{N\times m\cdot (N\times m+1)}{\left(1+\frac{K}{m}\right)^{N\times m+2}} \\
\end{split}
\end{equation*}

In [5]:
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative

We now want to value a CMS contract with payoff $g(K)$ at time T. Let $h(K) = \frac{g(K)}{\text{IRR}(K)}$. Differentiating, we get:

\begin{equation*}
  \begin{split}
    h(K) &= \frac{g(K)}{\mbox{IRR}(K)} \\
    h'(K) &= \frac{\mbox{IRR}(K)g'(K) - g(K)\mbox{IRR}'(K)}{\mbox{IRR}(K)^2} \\
    h''(K) &= \frac{\mbox{IRR}(K)g''(K)-\mbox{IRR}''(K)g(K) -2\cdot\mbox{IRR}'(K)g'(K)}{\mbox{IRR}(K)^2} \\
    &\;\;\;\;\;\;\;\;\;\;+
    \frac{2\cdot\mbox{IRR}'(K)^2g(K)}{\mbox{IRR}(K)^3}.
  \end{split}
\end{equation*}

In [6]:
def h_0(K, m, N):
    # implementation of h(K)
    value = g_0(K) / IRR_0(K, m, N)
    return value

def h_1(K, m, N):
    # implementation of h'(K) (1st derivative)
    firstDerivative = (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2
    return firstDerivative

def h_2(K, m, N):
    # implementation of h''(K) (2nd derivative)
    secondDerivative = ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) - 2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2
                        + 2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return secondDerivative

For CMS rate payoff, the payoff function is $g(F)$, and the static replication formula is:

  \begin{equation*}
    \begin{split}
      D(0,T) g(F) + \underbrace{\int_0^F h''(K) V^{rec}(K) dK}_\text{Put Integral} + \underbrace{\int_F^\infty h''(K) V^{pay}(K) dK}_\text{Call Integral}
    \end{split}
  \end{equation*}

In [7]:
# Define parameters
F, D_0T = fsr.loc[9, "FSR"], ois.loc[5, "Discount"]
alpha, beta, rho, nu = SABRdf.loc[9, 'Alpha'], 0.9, SABRdf.loc[9, 'Rho'], SABRdf.loc[9, 'Nu']
m, N, T, disc = 2, 10, 5, D_0T

# Terms
put_integral = quad(lambda x: h_2(x, m, N)*Black76(F, x, SABR(F, x, T, alpha, beta, rho, nu), T, D_0T, Type = 'Receiver'), 1e-6, F)
call_integral = quad(lambda x: h_2(x, m, N)*Black76(F, x, SABR(F, x, T, alpha, beta, rho, nu), T, D_0T, Type = 'Payer'), F, F*25)

# CMS valuation
cms_price_1 = D_0T * g_0(F) + put_integral[0] + call_integral[0]
print(cms_price_1)

0.2500422267269235


### Question 2

Suppose the payoff is now

$$
(CMS\ 10y^{\frac{1}{4}} - 0.04^{\frac{1}{2}})^+
$$

Let $F = \text{CMS}\ 10y$ and $g(K)$ be the payoff function. Differentiating, we get:

$$
\begin{align*}
g(K) \Rightarrow F^{\frac{1}{4}} - 0.2 &> 0 \\
F &> 0.2^{4} \\
F &> 0.0016 \\
\end{align*}
$$

Let L = 0.0016, which is the CMS caplet strike.

$$
\begin{align*}
\text{CMS Caplet} = V^{pay}(L)h'(L) + \int_L^{\infty} h''(K)V^{pay}(K)dK
\end{align*}
$$

In [8]:
# Define parameters
L = 0.2**4

# Terms
caplet = Black76(F, L, SABR(F, L, T, alpha, beta, rho, nu), T, D_0T, Type = 'Payer')*h_1(L, m, N)
call_integral_caplet = quad(lambda x: h_2(x, m, N)*Black76(F, x, SABR(F, x, T, alpha, beta, rho, nu), T, D_0T, Type = 'Payer'), L, L*25)

# CMS valuation
cms_price_2 = caplet + call_integral_caplet[0]
print(cms_price_2)

0.030343493389867346
